# Products

## Generating Products

When generating products for a source with multi-telescope observations associated with it, XGA will interact with each telescope specific software needed. Since different softwares are used, it is important to consider the subtlities and caveats associated with the different telescopes. In this tutorial we detail the considerations the user should take into account when comparing products from different telescopes. We do not go into full detail of the capabilities of the product related functions, for this please seek the other pages of the docs. 

In [ ]:
# Let's declare a source to demonstrate with
from astropy.units import Quantity
from xga.sources import GalaxyCluster
from xga.generate.esass import evtool_image, expmap, srctool_spectrum, srctool_lightcurve
from xga.generate.sas import evselect_image, eexpmap, evselect_spectrum, evselect_lightcurve

In [ ]:
src = GalaxyCluster(187.70527, 12.39243, 0.00377, 
                    name='Virgo', r500=Quantity(500, 'kpc'),
                    search_distance={'erosita': Quantity(3, 'deg'), 
                    'xmm': Quantity(30, 'arcmin')}
                    )

## Images

### eROSITA
Images are generated with different sizes depending on their origin, but all with a binning of 4.35 arcseconds per pixel. This was chosen to match the XCS binsize. 
* **Pointed** observations - The size of the image is 1.05 degrees, this is so the full eROSITA FOV (1.03 degrees) is captured.
* **Calibration Fields** (ie. eFEDS) - Images are 7.5 by 9 degrees in the x and y direction respectively. 
* **Other Calibration Fields** (ie eta Cha) - Images are 7 by 7 degrees.
* **eRASS sky tiles** - Images are 3.6 by 3.6 degrees, chosen to match the eROSITA team's definition of a skytile. This will have some overlapping areas between other skytiles, so the area of the sky is not completely unique.

In [ ]:
# Making images for eROSITA observations is done through the evtool_image function
evtool_image(src)

### XMM
All images are made with a binning of 4.35 arcseconds per pixel.

In [ ]:
evselect_image(src)

There is an additional optional argument in the XMM image generation function: 'add_expr'. This allows the user to add a string to the SAS expression keyword. 

In [ ]:
evselect_image(src, add_expr='')

## Exposure Maps
### eROSITA
XGA generates one merged exposure map for all seven telescope modules aboard eROSITA. Note that for non pointed osbervations, exposure map generation can take a longer time, as it has to calculate exposures for each snapshot of time, as the telescope slews across the sky. 

XGA also removes the 'REFYCRVL' keyword from the generated exposure map, as this get occupied with a nonsense value (as confirmed with the eROSITA team), which cannot be read into memory later. 

In [ ]:
expmap(src)

### XMM

In [ ]:
eexpmap(src)

## Spectra
### eROSITA
Spectra are generated differently depending on the source type and observing mode.

Source type:
* Spectra of **extended** sources are generated using the psftype=NONE so this does not apply any PSF corrections. The exttype parameter is set to MAP, this means that the spatial extent of the source is described by a detection map. This detection map is an image generated by XGA of the observation one is generating a spectrum for, the image is of the full eROSITA energy range (0.2 keV - 10 keV). This is then used for effective area calculations.
* Spectra of **point** sources are generated using the psftype='2D_PSF', and exttype='POINT'.

Observing Mode:
Since a source will move across the field of view as the telescope slews, the PSF and effective area of that source will vary. There are two parameters eSASS uses to control the accuracy of the spectra generated due to this variation. The first is **tstep**, which are the time bins that the PSF correction, vignetting correction and effective area are calculated for. The second is **xgrid**, which controls the sampling of the PSF, vignetting, and the effective area to perform the corrections with. Processing time is very sensitive to these parameters, so within XGA we have chosen these values to compromise between accuracy and computing time. 

* **Survey** mode - Tstep is set to 0.5s. Within this time the telescope will have scanned across 45 arcseconds (assuming the eRASS nominal scan speed) - which corresponds to about 5 detector pixels. 
* **Pointed** mode - Tstep is set to 100s. As the telescope is not moving with respect to the sky in this observing mode, we can increase the tstep, decrease processing time, and not sacrifice any spectral generation accuracy. 
* For both modes we set xgrid to be its default eSASS value (1.0). 


For both sources, the background spectrum are generated seperately from the source spectrum. This is done for two reasons, the first is so the background spectrum has its own ARF and RMF, which is appropriate for large sources. Secondly eSASS will use the GTIs of the source spectra for the background spectra - if generated together. If the source is a large extended source, this can have the effect of throwing away lots of usable data from the background. We therefore chose to generate them separately, so that we use the GTIs of the background region for background spectra generation. Lastly we increase the timestep to 2s for the background region, as this decreases the processing time, and less spatial accuracy is acceptable for background spectra. 

Spectra may be grouped by either a minimum count or signal to noise, this is implemented using Heasoft's ftgrouppha command. 

In [ ]:
# Making spectra for eROSITA observations is done through the srctool_spectrum function
srctool_spectrum(src, 'r500')

In [ ]:
# By default this generates a combined spectrum for all eROSITA instruments
# To generate each instrument separately use the combine_tm argument
srctool_spectrum(src, 'r500', combine_tm=False)

### XMM

## Lightcurves
### eROSITA
To generate eROSITA lightcurves, the srctool command is used. For all source types we set: 'exttype'='POINT', 'psftype'= '2D_PSF', tstep='0.5', 'lct'='REGULAR', and 'lc_gamma'='1.9'. 

In [ ]:
# Making lightcurves for eROSITA observations is done through the srctool_lightcurve function
srctool_spectrum(src, 'r500')

In [ ]:
# By default this generates a combined lightcurve for all eROSITA instruments
# To generate each instrument separately use the combine_tm argument
srctool_spectrum(src, 'r500', combine_tm=False)

Allowed pattern types can be controlled through the 'patt' argument. To see how to represent different patterns see: https://erosita.mpe.mpg.de/edr/DataAnalysis/prod_descript/EventFiles_edr.html 

In [ ]:
# Here we select all valid patterns (which is the XGA default)
srctool_spectrum(src, 'r500', patt=15)

### XMM

## Retrieving products

All of the product retrieval functions have a 'telescope' argument, allowing the user to only get products from a certain mission.

In [ ]:
# For example if we only wanted eROSITA images of our source
src.get_images(telescope='erosita')

## Viewing Images

#### eROSITA
Since eRASS is divided into large sky tiles, when viewing an image with XGA, it can be difficult to see the source within the full skytile image using the default arguments in the view() method. Here we reccommend some optional arguments to use to see a source in more detail.  

In [ ]:
# Retrieving the eROSITA images
images = src.get_images(telescope='erosita')

for img in images:
    # get_mask returns the source and background mask, 
    # so we use [0] index to get just the source mask
    mask = src.get_mask('r500', obs_id=img.obs_id, telescope='erosita')[0]
    # This will then show us the r500 region of the image
    img.view(mask=mask)